In [53]:
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from PIL import Image
from glob import glob
import time

In [54]:
import torch
import torch.nn as nn
class BMBaseModel(nn.Module):

    def __init__(self, network, num_classes):
        super(BMBaseModel, self).__init__()
        if network == "resnet":
            self.featurizer = torch.hub.load('pytorch/vision:v0.10.0', "resnet18", pretrained=True)
        elif network == "googlenet":
            self.featurizer = torch.hub.load('pytorch/vision:v0.10.0', "googlenet", pretrained=True)
        elif network == "inception":
            self.featurizer = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
        elif network == "googlenet":
            self.featurizer = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)
        #self.featurizer = self._freeze_layers(self.featurizer)
        self.mid_blocks = nn.Sequential(
                nn.Linear(1000, 2048),
                #nn.ReLU(),
                nn.Linear(2048, 1024),
                #nn.ReLU(),
                nn.Linear(1024, 256),
        )
        self.output_layer = nn.Linear(256, num_classes)

    def _freeze_layers(self, network):
        for param in network.parameters():
            param.requires_grad = False
        return network
    
    def forward(self, x):
        featurizer_op = self.featurizer(x)
        flattened_op = featurizer_op.view(featurizer_op.shape[0], -1)
        return self.output_layer(self.mid_blocks(flattened_op))
    
base_model = BMBaseModel("resnet", 21)

Using cache found in C:\Users\sr233/.cache\torch\hub\pytorch_vision_v0.10.0


In [55]:
exp1best_chkp = torch.load("E:\\bone-marrow cell classification\\results\\exp1\\exp1_best.pt")
base_model.load_state_dict(exp1best_chkp["model"])

<All keys matched successfully>

In [56]:
from torchcam.methods import SmoothGradCAMpp
cam_extractor1 = SmoothGradCAMpp(base_model)

In [57]:
from torchvision.transforms.functional import normalize, resize, to_pil_image
import matplotlib.pyplot as plt
from torchcam.utils import overlay_mask
from torchvision.transforms import transforms

In [58]:
from PIL import Image

In [59]:
def visualize_grad_cam(source, model,cam_extractor, save_dir):
  img = Image.open(source)
  img = img.resize((224, 224),Image.ANTIALIAS)
  img = transforms.ToTensor()(img).unsqueeze(0)
  out = model(img.cuda())
  activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)
  result = overlay_mask(to_pil_image(img.squeeze(0).cpu()), to_pil_image(activation_map[0].cpu(), mode='F'), alpha=0.5)
  #plt.imshow(result);plt.axis('off');plt.tight_layout()
  #plt.savefig(os.path.join(save_dir,source.split("\\")[-1]))
  result.save(os.path.join(save_dir,source.split("\\")[-1]))


In [60]:
exp1model = base_model.cuda()

In [61]:
test_files = sorted(glob("gradcam_test/*"))
save_dir = "exp_results/exp1"
for fp in test_files:
    print(fp)
    visualize_grad_cam(fp, exp1model, cam_extractor1, save_dir)

gradcam_test\ABE_00002.jpg


C:\Users\sr233\AppData\Local\Temp\ipykernel_12836\4050851166.py:3: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = img.resize((224, 224),Image.ANTIALIAS)


gradcam_test\ABE_00003.jpg
gradcam_test\ABE_00004.jpg
gradcam_test\ART_06027.jpg
gradcam_test\ART_06028.jpg
gradcam_test\ART_06029.jpg
gradcam_test\BAS_00012.jpg
gradcam_test\BAS_00013.jpg
gradcam_test\BAS_00014.jpg
gradcam_test\BLA_09012.jpg
gradcam_test\BLA_09013.jpg
gradcam_test\BLA_09014.jpg
gradcam_test\EBO_08027.jpg
gradcam_test\EBO_08028.jpg
gradcam_test\EBO_08029.jpg
gradcam_test\EOS_03012.jpg
gradcam_test\EOS_03013.jpg
gradcam_test\EOS_03014.jpg
gradcam_test\FGC_00039.jpg
gradcam_test\FGC_00040.jpg
gradcam_test\FGC_00041.jpg
gradcam_test\HAC_00012.jpg
gradcam_test\HAC_00013.jpg
gradcam_test\HAC_00014.jpg
gradcam_test\KSC_00017.jpg
gradcam_test\KSC_00018.jpg
gradcam_test\KSC_00019.jpg
gradcam_test\LYI_00035.jpg
gradcam_test\LYI_00036.jpg
gradcam_test\LYI_00037.jpg
gradcam_test\LYT_08019.jpg
gradcam_test\LYT_08020.jpg
gradcam_test\LYT_08021.jpg
gradcam_test\MMZ_02015.jpg
gradcam_test\MMZ_02016.jpg
gradcam_test\MMZ_02017.jpg
gradcam_test\MON_02039.jpg
gradcam_test\MON_02040.jpg
g